In [523]:
import pandas as pd
import numpy as np

In [524]:
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [525]:
df= pd.read_csv('covid_toy.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   age        100 non-null    int64  
 1   gender     100 non-null    object 
 2   fever      90 non-null     float64
 3   cough      100 non-null    object 
 4   city       100 non-null    object 
 5   has_covid  100 non-null    object 
dtypes: float64(1), int64(1), object(4)
memory usage: 4.8+ KB


In [526]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [527]:
df.value_counts('cough')

cough
Mild      62
Strong    38
Name: count, dtype: int64

In [528]:
X_train, X_test, y_train, y_test= train_test_split(df.iloc[:, :5], df.iloc[:, -1], test_size= 0.2, random_state=42)

In [529]:
X_train

,age,gender,fever,cough,city
55,81,Female,101.0,Mild,Mumbai
88,5,Female,100.0,Mild,Kolkata
26,19,Female,100.0,Mild,Kolkata
42,27,Male,100.0,Mild,Delhi
69,73,Female,103.0,Mild,Delhi
...,...,...,...,...,...
60,24,Female,102.0,Strong,Bangalore
71,75,Female,104.0,Strong,Delhi
14,51,Male,104.0,Mild,Bangalore
92,82,Female,102.0,Strong,Kolkata


In [530]:
# checking for blank values
df.isnull().sum()


age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

### TRANSFORMING DIFFERENT COLUMNS AND ADDING THEM INTO ONE TABLE

In [531]:
#filling blank values using simple imputer
si=SimpleImputer()
X_train_fever= si.fit_transform(X_train[['fever']])

#imputing test set
X_test_fever= si.transform(X_test[['fever']])

In [532]:
# Ordinal encoding on cough

oe= OrdinalEncoder(categories= [['Mild', 'Strong']])
X_train_cough=oe.fit_transform(X_train[['cough']])

#encoding test set
X_test_cough= oe.transform(X_test[['cough']])

In [533]:
# One Hot encoding on gender and city

ohe= OneHotEncoder(drop='first',sparse_output=False)
cols=['gender', 'city']

X_train_gender_city= ohe.fit_transform(X_train[cols])

X_test_gender_city= ohe.transform(X_test[cols])


In [534]:
# getting age out

X_train_age= X_train.drop(columns=['gender','fever','cough','city']).values


X_test_age= X_test.drop(columns=['gender','fever','cough','city']).values


In [535]:
#adding all these columns to form a dataset

X_train_transformed = np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough),axis=1)

X_test_transformed = np.concatenate((X_test_age,X_test_fever,X_test_gender_city,X_test_cough),axis=1)

### DOING THE SAME THING BUT  USING THE COLUMN TRANSFORMER

In [536]:
from sklearn.compose import ColumnTransformer

transformer = ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(sparse_output=False,drop='first'),['gender','city'])
],remainder='passthrough')

In [537]:
transformer.fit_transform(X_train).shape

(80, 7)